In [1]:
import pandas as pd
import numpy as np
from nltk.stem.porter import re
from pycaret.regression import setup, compare_models, blend_models,tune_model,predict_model,get_config, finalize_model

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


## 메뉴 토큰화

### 테스트 데이터와 학습 데이터 메뉴 병합
- 현재 학습 데이터의 메뉴와 테스트 데이터의 메뉴가 동일하지 않기 때문에 통일시키는 과정이 필요
- 합집합을 사용할지 교집합을 사용할지 추후 결정

In [4]:
menu_train = train[['조식메뉴','중식메뉴','석식메뉴']]
menu_test = test[['조식메뉴','중식메뉴','석식메뉴']]
print(menu_train.shape)
print(menu_test.shape)

(1205, 3)
(50, 3)


In [5]:
menu_total = pd.concat([menu_train,menu_test],axis=0)
print(menu_total.shape)

(1255, 3)


#### 전처리 함수

In [6]:
def clean_split(df):
    df = df.split()
    for j in df:
        if '(' in j:
            del df[df.index(j)]
    for i in df:
        if '쌀밥' in i:
            del df[df.index(i)] 
    for q in df:
        if '김치' in q:
            del df[df.index(q)]
    
    return df

In [7]:
menu_total['조식메뉴_prepro'] = menu_total['조식메뉴'].apply(clean_split)
menu_total['중식메뉴_prepro'] = menu_total['중식메뉴'].apply(clean_split)
menu_total['석식메뉴_prepro'] = menu_total['석식메뉴'].apply(clean_split)

In [8]:
menu_total.head()

,조식메뉴,중식메뉴,석식메뉴,조식메뉴_prepro,중식메뉴_prepro,석식메뉴_prepro
0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...","[모닝롤/찐빵, 우유/두유/주스, 계란후라이, 된장찌개, 쥐어채무침]","[오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트]","[육개장, 자반고등어구이, 두부조림, 건파래무침]"
1,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...","[모닝롤/단호박샌드, 우유/두유/주스, 계란후라이, 호박젓국찌개, 시래기조림]","[가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추겉절이]","[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나]"
2,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...","[모닝롤/베이글, 우유/두유/주스, 계란후라이, 콩나물국, 느타리호박볶음]","[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트]","[청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음]"
3,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...","[모닝롤/토마토샌드, 우유/두유/주스, 계란후라이, 근대국, 멸치볶음]","[쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트]","[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클]"
4,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...","[모닝롤/와플, 우유/두유/주스, 계란후라이, 재첩국, 방풍나물]","[떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트]","[차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침]"


In [9]:
lunch = menu_total.중식메뉴_prepro.tolist()
dinner = menu_total.석식메뉴_prepro.tolist()

In [10]:
lunch_list = []
for i in lunch:
    if len(i) == 0:
        continue
    else:
        string = i[0]
    for w in i[1:]:
        string += " "
        string += w
    lunch_list.append(string)
lunch_list

['오징어찌개 쇠불고기 계란찜 청포묵무침 요구르트',
 '가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이',
 '카레덮밥 팽이장국 치킨핑거 쫄면야채무침 견과류조림 요구르트',
 '쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트',
 '떡국 돈육씨앗강정 우엉잡채 청경채무침 요구르트',
 '시래기국 훈제오리구이 도토리묵무침 쌈무/양파절임 요구르트',
 '꽃게탕 돈육굴소스볶음 옥수수전 유채나물 요구르트',
 '시금치국 닭감자조림 연두부*양념장 콩나물무침 요구르트',
 '쇠고기무국 탕수어 오징어숙회무침 취나물 요구르트',
 '냉이된장국 쇠고기장조림 통도라지구이 치커리무침 요구르트',
 '낙지비빔밥 유부장국 고구마치즈구이 해초무침 요구르트 요구르트',
 '대구찌개 마늘간장치킨 새송이버섯볶음 돌나물무침/치킨무 요구르트',
 '쇠고기느타리국 갈치구이 호박꼬지&고사리&무나물 파래김*양념장 부럼/요구르트',
 '콩나물국 돈육간장볶음 버섯잡채 꽃상추무침 요구르트',
 '비빔밥 팽이장국 소세지피망볶음 맛탕 귤 요구르트',
 '북어계란국 닭갈비 새싹두부구이 곰피초장 요구르트',
 '수제비국 오징어볶음 계란찜 도라지초무침 요구르트',
 '얼갈이된장국 제육볶음 늙은호박전 치커리생채 요구르트',
 '쇠고기미역국 주꾸미볶음 맛살계란말이 아삭고추무침',
 '근대된장국 탕수육 새송이버섯조림 무생채 요구르트',
 '부대찌개 가자미튀김 메추리알조림 열무된장나물 깍두기',
 '콩나물국 닭갈비 모둠묵*양념장 톳두부무침',
 '차돌박이찌개 소세지구이 풋마늘초무침 깍두기',
 '북어국 돈육장조림 홍어무침 시금치나물',
 '고추장찌개 닭데리야끼조림 해물파전 취나물',
 '배추된장국 수제돈가스 탕평채 쫄면무침',
 '어묵국 돈육고추장볶음 모둠양채쌈*쌈장 콩나물파채무침',
 '쑥국 소불고기 골뱅이무침*소면 마늘쫑볶음',
 '곤드레밥 미니채소떡갈비 두부계란부침 무나물',
 '사골우거지국 탕수어 파래김*양념장 깻순나물 깍두기',
 '시금치국 훈제오리구이 연근땅콩조림 쌈무/부추생채',
 '꽃게탕 버섯불고기

In [11]:
dinner_list = []
for i in dinner:
    if len(i) == 0:
        i = ['.']
        string = i[0]
    else:
        string = i[0]
    for w in i[1:]:
        string += " "
        string += w
    dinner_list.append(string)
dinner_list

['육개장 자반고등어구이 두부조림 건파래무침',
 '콩나물밥*양념장 어묵국 유산슬 아삭고추무침 바나나',
 '청국장찌개 황태양념구이 고기전 새송이버섯볶음',
 '미니김밥*겨자장 우동 멕시칸샐러드 군고구마 무피클',
 '차돌박이찌개 닭갈비 감자소세지볶음 콩나물무침',
 '참치회덮밥 맑은국 군만두 과일샐러드 락교',
 '미니함박 어묵볶음 물파래무침 깍두기',
 '홍합미역국 임연수구이 브로컬리초장',
 '된장찌개 쇠불고기 해파리겨자채 봄동무침',
 '볶음밥*자장소스 맑은국 새우또띠아 쨔샤이무침 요플레',
 '순두부찌개 명태조림 감자채볶음 쌈만두',
 '설렁탕 야채스틱 바나나 석박지',
 '옹심이만두국 떡갈비조림 청포묵무침 부추오이생채',
 '콩비지찌개 해물까스&타르타르소스 볶음쌀국수 미역레몬초무침 통배추겉절이',
 '열무된장국 매운돼지갈비찜 멸치호두볶음 숙주나물',
 '김밥 잔치국수 교자만두 찜) 멕시칸샐러드 무피클',
 '닭개장 삼치구이 꽃맛살볶음 청경채사과무침',
 '버섯들깨탕 꽁치한마리구이 두부양념조림 구이김',
 '된장찌개 버섯불고기 콩나물겨자채 양념깻잎지',
 '멸치주먹밥 우동 떡볶이 군고구마 단무지',
 '오징어국 미트볼조림 옥수수전 부추생채',
 '대구찌개 돈육굴소스볶음 감자채볶음 물파래무침',
 '카레덮밥 가쯔오장국 수제고로케*케찹 과일샐러드 오복지',
 '육개장 생선까스*탈탈소스 야채계란찜 콩나물무침',
 '볶음밥 짬뽕 개성감자만두 타코야끼 꼬들단무지',
 '연두부탕 코다리무조림 고추잡채*꽃빵 봄동겉절이',
 '달래된장찌개 간장찜닭 통도라지구이 참나물생채',
 '짜장잡채덮밥 계란파국 참치야채전 꽃맛살샐러드 오이무침',
 '뼈해장국 해물청경채볶음 버섯메밀전 야채스틱*쌈장',
 '토마토스파게티 양송이스프 미니햄버거 단호박범벅 무피클/음료',
 '닭곰탕 임연수찜 어묵곤약볶음 돌나물초장 석박지',
 '수제비국 아귀콩나물찜 새송이버섯볶음 봄동겉절이',
 '굴소스파인볶음밥 가쯔오장국 찹쌀순대볶음 궁중떡찜 두반장가지나물',
 '알탕 동그랑땡전 마파두부 시금치나물

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
lunch_matrix = vectorizer.fit_transform(lunch_list)
lunch_df = pd.DataFrame(lunch_matrix.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
dinner_matrix = vectorizer.fit_transform(dinner_list)
dinner_df = pd.DataFrame(dinner_matrix.toarray(), columns=vectorizer.get_feature_names())

In [13]:
lunch_df

,d오리엔탈d,la갈비구이,new,가래떡구이,가래떡돼지갈비찜,가래떡츄러스,가자미구이,가자미무조림,가자미양념찜,가자미엿장구이,가자미엿장조림,가자미유린기,가자미조림,가자미카레튀김,가자미튀김,가지고추장무침,가지구이,가지나물,가지나물무침,가지두반장볶음,가지무침,가지볶음,가지완자튀김,가지찜,가쯔오국,가쯔오장국,간장,간장깻잎지,간장돼지갈비찜,간장두부조림,간장마늘치킨,간장양념,간장찜닭,간장치킨,간장파닭,갈릭돈가스,갈릭돈까스,갈릭버섯탕수,갈릭순살치킨,갈비만두,갈비탕,갈비통통만두,갈치감자조림,갈치구이,갈치무조림,갈치양념조림,갈치조림,감귤d,감자고구마샐러드,감자고추장찌개,감자국,감자그라탕,감자만두,감자범벅,감자볶음,감자비엔나볶음,감자샐러드,감자수제비국,감자양파국,감자전,감자조림,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자치즈구이,감자프리타타,강된장,강된장찌개,강정소스,개성감자만두,개성식메밀부침개,건강비빔밥,건다래순볶음,건새우마늘쫑볶음,건새우무국,건새우무채국,건새우미역국,건새우아욱국,건새우호박볶음,건새우호박채전,건취나물볶음,건파래무침,검은깨올방개묵무침,검정콩조림,견과류마카로니범벅,견과류멸치볶음,견과류샐러드,견과류연근조림,견과류조림,견과쌈장,경상도식소고기국,계란국,계란말이,계란버섯장조림,계란장조림,계란찜,계란채소볶음밥,계란파국,계란후라이,고구마고로케,고구마그라탕,고구마범벅,고구마순나물,고구마순무침,고구마순볶음,고구마줄기볶음,고구마치즈구이,고구마치즈돈까스,고구마치즈빵,고구마튀김,고구마함박스테이크,고기듬뿍카레라이스,고기완자전,고기전,고등어구이,고등어조림,고등어카레구이,고사리,고사리육개장,고추,고추가루,고추간장지,고추잎나물,고추잡채,고추장감자조림,고추장고구마순무침,고추장누들떡볶이,고추장불고기,고추장찌개,고추지,고추지무침,고추튀김,고춧잎볶음,곤드레밥,곤약메추리알조림,곤약야채무침,곤약어묵볶음,곤약흑임자무침,골뱅이무침,골뱅이소면무침,골뱅이야채무침,골뱅이채소무침,곰취,곰피초장,과일,과일그라탕,과일샐러드,과일요거트샐러드,과일탕수육,교촌간장치킨,구운채소,구이김,군고구마,군만두,굴떡국,굴미역국,굴비구이,궁중떡볶음,궁중떡볶이,궁중떡찜,그린샐러드,근대국,근대나물,근대된장국,김구이,김말이강정,김말이튀김,김밥볶음밥,김칫국,깍두기,깍둑오이초무침,깐풍기,깐풍두부,깐풍연근,깐풍육,깻순나물,깻잎,깻잎무쌈,깻잎순무침,깻잎쌈,깻잎양념지,깻잎완자전,깻잎전,깻잎지,깻잎찜,깻잎통닭,꼬들단무지무침,꼬막미나리초무침,꼬시래기무초무침,꼬시래기무침,꼬지삼색전,꼬지어묵탕,꽁치구이,꽁치레몬구이,꽁치오븐구이,꽁치와사비구이,꽁치한마리구이,꽁치한마리레몬구이,꽃게된장국,꽃게된장찌개,꽃게찌개,꽃게탕,꽃맛살과일샐러드,꽃맛살볶음,꽃맛살샐러드,꽃맛살오리엔탈샐러드,꽃맛살햄볶음,꽃빵,꽃상추겉절이,꽃상추무침,꽈리고추감자조림,꽈리고추메추리알조림,꽈리고추멸치볶음,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,나가사끼짬뽕국,나물비빔밥,나주곰탕,나쵸콥샐러드,낙지볶음,낙지볶음밥,낙지비빔밥,날치알계란찜,냉메밀소바,냉모밀국수,냉이국,냉이나물무침,냉이된장국,냉이된장찌개,냉이콩나물국,냉족발야채무침,너비아니구이,노각무침,노각생채,녹두전,누룽지탕수육,눈꽃치즈샐러드,느타리버섯국,느타리버섯볶음,늙은호박전,다대기,다슬기아욱국,다시마,다시마쌈,단무지,단무지락교무침,단무지무침,...,치즈닭갈비,치즈불닭,치즈함박스테이크,치커리들깨무침,치커리만다린무침,치커리만다린샐러드,치커리무침,치커리사과무침,치커리사과생채,치커리사과유자청무침,치커리생채,치커리오이무침,치커리유자무침,치커리유자샐러드,치커리유자생채,치커리유자청무침,치커리유자청생채,치킨까스,치킨너겟,치킨무,치킨샐러드,치킨소스,치킨커틀렛,치킨텐더,치킨텐더샐러드,치킨핑거,칠리,칠리드레질,칠리새우,칠리소스,칠리탕수육,카레감자채볶음,카레닭볶음,카레닭찜,카레덮밥,카레라이스,카프레제샐러드,칼국수,캔꽁치튀김,케일,케일숙쌈,케찹,코다리강정,코다리양념조림,코다리조림,콘샐러드,콘소스d,콘슬로우,콘치즈오븐구이,콜리샐러드,콥샐러드,콩가루배추국,콩가루배추된장국,콩가루배춧국,콩나물겨자채,콩나물겨자채무침,콩나물국,콩나물김칫국,콩나물냉국,콩나물냉채,콩나물동태찜,콩나물두루치기,콩나물맑은국,콩나물맛살냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물부추무침,콩나물불고기,콩나물잡채,콩나물제육볶음,콩나물파채,콩나물파채무침,콩나물파채불고기,콩나물파채절이,콩비지찌개,콩샐러드,콩소스d,콩자반,콩조림,크란치바,크래미숙주무침,크래미해초무침,크레미계란말이,크루통샐러드,크리스마스케익,크림새우,크림스프,키위d,키위요거트d,타꼬야끼,타래과,타르,타르소스,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,탱크보이,토란국,토란탕,토마토,토마토계란볶음,토마토두부카프레제,토마토살사소스,토마토샐러드,토마토스크램블,토마토스파게티,토마토프리타타,톳두부무침,톳무침,통감자오븐구이,통계란꼬치어묵탕,통도라지고추장구이,통도라지구이,통들깨부추무침,통오이고추무침,파닭,파래김,파스타샐러드,파인d,파인애플,파인애플d,파인애플볶음밥,파인요거트d,파채,파채상추무침,파채콩나물무침,파프리카감자채볶음,파프리카계란말이,파프리카계란전,파프리카잡채,파프리카해초무침,팝콘치킨,팽이된장국,팽이무국,팽이버섯국,팽이장국,펜네파스타샐러드,포도d,포도주스,표고돈육탕수,표고버섯탕수육,푸실리파스타샐러드,풋고추,풋고추양파쌈장무침,풋마늘대무침,풋마늘초무침,풍기샐러드,프로바이오틱,피클,하루나겉절이,하와이안샐러드,하와이언함박스테이크,하이소스,한방갈비탕,한방설렁탕,한방소갈비찜,한식잡채,할라피뇨채소피클,함박스테이크,핫도그,해물겨자냉채,해물굴소스볶음,해물까스,해물누룽지탕,해물동그랑땡,해물동그랑땡채소볶음,해물돼지갈비찜,해물된장찌개,해물볶음우동,해물부추전,해물섞어찜,해물수제비국,해물순두부찌개,해물완자전,해물잡채,해물전,해물청경채볶음,해물콩나물찜,해물탕,해물파전,해장탕,해초무침,해초샐러드,해파리겨자채,해파리냉채,해파리무침,햄감자채볶음,햄계란말이,햄피망볶음,허니버터치킨,허니순살치킨,호두견과류강정,호박고추장찌개,호박꼬지,호박나물,호박된장국,호박된장찌개,호박맑은국,호박부추전,호박새우젓볶음,호박숙,호박잎,호박잎된장국,호박잎된장찌개,호박잎쌈,호박잎찌개,호박전,호박채나물,호박채나물볶음,홍삼,홍어무침,홍어미나리초무침,홍어채무침,홍초콩나물국,홍합국,홍합살무국,홍합살미역국,홍합탕,황도d,황도샐러드,황도요거트d,황태국,황태맑은국,황태미역국,황태양념구이,황태채국,황태채마늘쫑무침,황태콩나물해장국,황태포무침,후라이드,후르츠탕수육,훈제오리,훈제오리고추장볶음,훈제오리구이,훈제오리냉채,훈제오리단호박볶음,훈제오리마늘볶음,훈제오리볶음,흑임자d,흑임자드레싱,흑임자연근샐러드,히레카츠
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.364542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [14]:
dinner_df

,가래떡오븐구이,가쓰오장국,가자미구이,가자미양념조림,가자미양념찜,가자미엿장구이,가자미유린기,가자미찜,가자미카레튀김,가자미튀김,가정의날,가지나물,가지된장무침,가지마파두부,가지무침,가지탕수,가쯔오국,가쯔오장국,간장,간장계란장,간장깻잎지,간장마늘치킨,간장불고기,간장찜닭,간장치킨,갈릭파닭,갈비만두,갈치구이,갈치무조림,갈치양념조림,갈치조림,갈치카레구이,갈치튀김,감귤쥬스,감자간장조림,감자고추장찌개,감자국,감자만두,감자만두국,감자버터구이,감자범벅,감자비엔나볶음,감자샐러드,감자소세지볶음,감자양념조림,감자양파국,감자전,감자찌개,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자채햄볶음,감자치즈구이,감자치즈오븐구이,감자카레볶음,감자튀김,감자프리타타,감자햄조림,강된장,개성감자만두,건도토리묵파프리카볶음,건새우마늘쫑볶음,건새우무나물,건새우아욱국,건새우아욱된장국,건새우호박채전,건파래무침,건파래볶음,검정콩조림,겨울초겉절이,겨자장,견과류조림,계란국,계란김밥,계란떡볶이,계란말이,계란볶음밥,계란야채말이,계란장조림,계란찜,계란파국,계란후라이,계발의,고갈비구이,고구마그라탕,고구마까스,고구마떡볶이,고구마맛탕,고구마범벅,고구마샐러드,고구마순나물,고구마순들깨볶음,고구마순볶음,고구마연근맛탕,고구마오븐구이,고구마치즈구이,고구마치즈돈까스,고구마튀김,고기전,고기짬뽕국,고들빼기무침,고등어구이,고등어양념구이,고등어자반찜,고등어조림,고등어카레구이,고르곤졸라피자,고사리볶음,고추,고추가루,고추잡채,고추잡채덮밥,고추장멸치볶음,고추장불고기,고추장찌개,고추지무침,고추짜장,고추튀김,고춧잎무침,곤드레밥,골뱅이무침,골뱅이채소무침,과일,과일샐러드,과일요거트샐러드,과일주스,과일탕수육,과일플레인샐러드,교자만두,구슬떡볶이,구운채소,구이김,국내산,국물떡볶이,국산,군고구마,군만두,굴미역국,굴비구이,굴소스볶음밥,굴소스파인볶음밥,굴소스해물볶음밥,궁중떡볶음,궁중떡볶이,궁중떡찜,그린빈베이컨볶음,그린요거트샐러드,근대고추장무침,근대국,근대나물무침,근대된장국,근대두부된장국,김가루잔파무침,김계란말이,김구이,김말이,김말이강정,김말이튀김,김밥,김밥볶음밥,김자반,김자반볶음,김주먹밥,김칫국,까르보나라떡볶이,깍두기,깍둑오이초무침,깐쇼새우,깐풍기,깐풍연근,깐풍육,깨찰빵,깻순나물,깻잎고기전,깻잎쌈,깻잎양념찜,깻잎전,깻잎지,깻잎찜,꼬들단무지,꼬들단무지무침,꼬마김밥,꼬막찜,꼬막채소무침,꼬시래기무침,꼬시래기초무침,꼬지어묵국,꼬지어묵우동,꼬지어묵탕,꼬치어묵국,꼬치어묵떡볶이,꼬치어묵매운탕,꼬치어묵탕,꽁보리밥,꽁치감자조림,꽁치구이,꽁치양파조림,꽁치오븐구이,꽁치한마리구이,꽁치허브구이,꽃게탕,꽃맛살볶음,꽃맛살샐러드,꽃빵,꽃빵튀김,꽃상추무침,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,꽈배기도너츠,꿔바로우탕수육,나가사끼짬뽕,나가사끼짬뽕국,나가사키면,나가사키짬뽕국,나초콥샐러드,낙지덮밥,낙지미나리볶음,낙지볶음,낙지볶음덮밥,낙지비빔밥,낙지젓무침,날치알계란찜,날치알볶음밥,납작군만두,내사랑포도,냉메밀소바,냉모밀국수,냉이국,냉이김칫국,냉이된장국,냉이된장찌개,너비아니,노가리고추조림,노각무침,녹두빈대떡,녹두전,누룽지탕수육,뉴욕핫도그,느타리버섯국,느타리버섯볶음,느타리호박볶음,늙은호박전,다슬기아욱국,다시마채무초무침,단감치커리무침,단무지,단무지무침,...,청양멸치주먹밥,청양콩나물국,청양해물파전,청포도,청포도피클,청포묵무침,청포묵야채무침,초간장,초계국수,초장,추가밥,춘권,춘장돈육볶음,춘전닭갈비,춘천닭갈비,충무김밥,취나물,취나물된장무침,취나물된장볶음,취나물땅콩무침,취나물무침,취나물볶음,츄러스,치즈계란말이,치즈계란찜,치즈닭갈비,치즈시즈닝치킨,치즈함박스테이크,치커리귤무침,치커리깨소스무침,치커리단감무침,치커리무침,치커리배생채,치커리사과무침,치커리사과생채,치커리생채,치커리유자무침,치커리유자청무침,치킨까스,치킨마요덮밥,치킨무,치킨샐러드,치킨퀘사디야,치킨텐더,치킨핑거,칠리s,칠리미트볼조림,칠리베이비크랩,칠리소스,카레감자채볶음,카레덮밥,카레돈까스정식,카레라이스,카레찜닭,카레홍합찜,칼국수,컬리플라워샐러드,컵라면,컵주스,케이준샐러드,케이준치킨샐러드,케일숙쌈,케찹,코다리강정,코다리무조림,코다리양념조림,코다리엿장조림,코다리조림,코코넛새우튀김,코코뱅,콘샐러드,콘슬로우,콘치즈,콘치즈구이,콘치즈오븐구이,콤비네이션피자,콥샐러드,콩가루배추국,콩국수,콩나물간장볶음,콩나물겨자채,콩나물국,콩나물굴소스볶음,콩나물냉국,콩나물냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물불고기,콩나물파채무침,콩비지찌개,콩자반,콩조림,퀘사디아,크래미오이보트샐러드,크랜베리단호박샐러드,크랜베리멸치볶음,크림새우,크림소스,크림스프,크림카레우동,클로렐라,키위그린샐러드,키위드레싱샐러드,타꼬야끼,타르타르,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,토란탕,토마토,토마토계란찜,토마토설탕절인,토마토스파게티,토마토시금치달걀볶음,톳두부무침,통고추쌈장무침,통도라지구이,통마늘너비아니조림,통배추겉절이,통새우김밥,통새우또띠아,투움바스파게티,파닭,파래김,파스타샐러드,파인애플볶음밥,파인애플주스,파채,파프리카해초무침,팝콘치킨,팥찜,팽이가쯔오장국,팽이버섯국,팽이버섯부추무침,팽이버섯장국,팽이버섯채소전,팽이장국,페퍼로니치즈피자,펜네베이컨샐러드,포기김치,포도,포도주스,포테이토오븐구이,폭탄주먹밥,표고버섯탕수육,푸딩,푸실리샐러드,풋고추,풋고추양파무침,풋고추튀김,풋마늘대무침,풋마늘초무침,풍기샐러드,피크닉,피클,하루나무침,하루야채주스,하와이안샐러드,하와이안필라프,한식잡채,할라피뇨,함박스테이크,핫도그,해라피겨자채,해물가스,해물겨자냉채,해물굴소스볶음밥,해물까스,해물누룽지,해물누룽지탕,해물동그랑땡,해물동그랑땡전,해물된장찌개,해물떡볶이,해물볶음,해물볶음밥,해물볶음우동,해물부추전,해물손수제비,해물수제비,해물순두부국,해물순두부찌개,해물우동볶음,해물전,해물짜장면,해물짬뽕,해물짬뽕국,해물청경채볶음,해물칼국수,해물파전,해쉬포테이토,해초레몬무침,해초무침,해초배무침,해파리겨자채,해파리냉채,햄감자볶음,햄감자채볶음,햄계란말이,햄맛살볶음,햄볶음밥,햄전,햄치즈또띠아,허니버터옥수수,허니슈스트링감자,호박나물,호박반달나물,호박부추전,호박새우젓국,호박잎된장찌개,호박잎찌개,호박젓국찌개,호박채나물,호빵,호주,홍시드레싱샐러드,홍어채무침,홍합무우국,홍합미역국,홍합짬뽕국,홍합탕,황태국,황태국수,황태무채국,황태미역국,황태양념구이,황태해장국,회오리감자,후난식볶음밥,후르츠탕수육,후리가케덮밥,훈제오리,훈제오리구이,훈제오리냉채,훈제오리떡볶음,훈제오리볶음,훈제오리불고기,흑미밥,흑임자곤약샐러드,흑임자드레싱,흑임자샐러드,흑임자시금치샐러드
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.549609,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [15]:
lunch_df_f = pd.concat([train,lunch_df[:1205],pd.get_dummies(train['요일'])],axis=1).drop(['요일','조식메뉴','중식메뉴','석식메뉴','석식계','일자'],axis=1)
# 신메뉴를 나타내는 'new'를 변수로서 넣을지 결정
lunch_df_test_f = pd.concat([test,lunch_df[1205:].reset_index(drop=True),pd.get_dummies(test['요일'])],axis=1).drop(['요일','조식메뉴','중식메뉴','석식메뉴','일자'],axis=1)


dinner_df_f = pd.concat([train,dinner_df[:1205],pd.get_dummies(train['요일'])],axis=1).drop(['요일','조식메뉴','중식메뉴','석식메뉴','중식계','일자'],axis=1)
dinner_df_test_f = pd.concat([test,dinner_df[1205:].reset_index(drop=True),pd.get_dummies(test['요일'])],axis=1).drop(['요일','조식메뉴','중식메뉴','석식메뉴','일자'],axis=1)

In [16]:
lunch_df_f.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,d오리엔탈d,la갈비구이,new,가래떡구이,가래떡돼지갈비찜,가래떡츄러스,가자미구이,가자미무조림,가자미양념찜,가자미엿장구이,가자미엿장조림,가자미유린기,가자미조림,가자미카레튀김,가자미튀김,가지고추장무침,가지구이,가지나물,가지나물무침,가지두반장볶음,가지무침,가지볶음,가지완자튀김,가지찜,가쯔오국,가쯔오장국,간장,간장깻잎지,간장돼지갈비찜,간장두부조림,간장마늘치킨,간장양념,간장찜닭,간장치킨,간장파닭,갈릭돈가스,갈릭돈까스,갈릭버섯탕수,갈릭순살치킨,갈비만두,갈비탕,갈비통통만두,갈치감자조림,갈치구이,갈치무조림,갈치양념조림,갈치조림,감귤d,감자고구마샐러드,감자고추장찌개,감자국,감자그라탕,감자만두,감자범벅,감자볶음,감자비엔나볶음,감자샐러드,감자수제비국,감자양파국,감자전,감자조림,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자치즈구이,감자프리타타,강된장,강된장찌개,강정소스,개성감자만두,개성식메밀부침개,건강비빔밥,건다래순볶음,건새우마늘쫑볶음,건새우무국,건새우무채국,건새우미역국,건새우아욱국,건새우호박볶음,건새우호박채전,건취나물볶음,건파래무침,검은깨올방개묵무침,검정콩조림,견과류마카로니범벅,견과류멸치볶음,견과류샐러드,견과류연근조림,견과류조림,견과쌈장,경상도식소고기국,계란국,계란말이,계란버섯장조림,계란장조림,계란찜,계란채소볶음밥,계란파국,계란후라이,고구마고로케,고구마그라탕,고구마범벅,고구마순나물,고구마순무침,고구마순볶음,고구마줄기볶음,고구마치즈구이,고구마치즈돈까스,고구마치즈빵,고구마튀김,고구마함박스테이크,고기듬뿍카레라이스,고기완자전,고기전,고등어구이,고등어조림,고등어카레구이,고사리,고사리육개장,고추,고추가루,고추간장지,고추잎나물,고추잡채,고추장감자조림,고추장고구마순무침,고추장누들떡볶이,고추장불고기,고추장찌개,고추지,고추지무침,고추튀김,고춧잎볶음,곤드레밥,곤약메추리알조림,곤약야채무침,곤약어묵볶음,곤약흑임자무침,골뱅이무침,골뱅이소면무침,골뱅이야채무침,골뱅이채소무침,곰취,곰피초장,과일,과일그라탕,과일샐러드,과일요거트샐러드,과일탕수육,교촌간장치킨,구운채소,구이김,군고구마,군만두,굴떡국,굴미역국,굴비구이,궁중떡볶음,궁중떡볶이,궁중떡찜,그린샐러드,근대국,근대나물,근대된장국,김구이,김말이강정,김말이튀김,김밥볶음밥,김칫국,깍두기,깍둑오이초무침,깐풍기,깐풍두부,깐풍연근,깐풍육,깻순나물,깻잎,깻잎무쌈,깻잎순무침,깻잎쌈,깻잎양념지,깻잎완자전,깻잎전,깻잎지,깻잎찜,깻잎통닭,꼬들단무지무침,꼬막미나리초무침,꼬시래기무초무침,꼬시래기무침,꼬지삼색전,꼬지어묵탕,꽁치구이,꽁치레몬구이,꽁치오븐구이,꽁치와사비구이,꽁치한마리구이,꽁치한마리레몬구이,꽃게된장국,꽃게된장찌개,꽃게찌개,꽃게탕,꽃맛살과일샐러드,꽃맛살볶음,꽃맛살샐러드,꽃맛살오리엔탈샐러드,꽃맛살햄볶음,꽃빵,꽃상추겉절이,꽃상추무침,꽈리고추감자조림,꽈리고추메추리알조림,꽈리고추멸치볶음,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,나가사끼짬뽕국,나물비빔밥,나주곰탕,나쵸콥샐러드,낙지볶음,낙지볶음밥,낙지비빔밥,날치알계란찜,냉메밀소바,냉모밀국수,냉이국,냉이나물무침,냉이된장국,냉이된장찌개,냉이콩나물국,냉족발야채무침,너비아니구이,노각무침,노각생채,녹두전,누룽지탕수육,눈꽃치즈샐러드,느타리버섯국,느타리버섯볶음,늙은호박전,다대기,...,치커리만다린샐러드,치커리무침,치커리사과무침,치커리사과생채,치커리사과유자청무침,치커리생채,치커리오이무침,치커리유자무침,치커리유자샐러드,치커리유자생채,치커리유자청무침,치커리유자청생채,치킨까스,치킨너겟,치킨무,치킨샐러드,치킨소스,치킨커틀렛,치킨텐더,치킨텐더샐러드,치킨핑거,칠리,칠리드레질,칠리새우,칠리소스,칠리탕수육,카레감자채볶음,카레닭볶음,카레닭찜,카레덮밥,카레라이스,카프레제샐러드,칼국수,캔꽁치튀김,케일,케일숙쌈,케찹,코다리강정,코다리양념조림,코다리조림,콘샐러드,콘소스d,콘슬로우,콘치즈오븐구이,콜리샐러드,콥샐러드,콩가루배추국,콩가루배추된장국,콩가루배춧국,콩나물겨자채,콩나물겨자채무침,콩나물국,콩나물김칫국,콩나물냉국,콩나물냉채,콩나물동태찜,콩나물두루치기,콩나물맑은국,콩나물맛살냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물부추무침,콩나물불고기,콩나물잡채,콩나물제육볶음,콩나물파채,콩나물파채무침,콩나물파채불고기,콩나물파채절이,콩비지찌개,콩샐러드,콩소스d,콩자반,콩조림,크란치바,크래미숙주무침,크래미해초무침,크레미계란말이,크루통샐러드,크리스마스케익,크림새우,크림스프,키위d,키위요거트d,타꼬야끼,타래과,타르,타르소스,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,탱크보이,토란국,토란탕,토마토,토마토계란볶음,토마토두부카프레제,토마토살사소스,토마토샐러드,토마토스크램블,토마토스파게티,토마토프리타타,톳두부무침,톳무침,통감자오븐구이,통계란꼬치어묵탕,통도라지고추장구이,통도라지구이,통들깨부추무침,통오이고추무침,파닭,파래김,파스타샐러드,파인d,파인애플,파인애플d,파인애플볶음밥,파인요거트d,파채,파채상추무침,파채콩나물무침,파프리카감자채볶음,파프리카계란말이,파프리카계란전,파프리카잡채,파프리카해초무침,팝콘치킨,팽이된장국,팽이무국,팽이버섯국,팽이장국,펜네파스타샐러드,포도d,포도주스,표고돈육탕수,표고버섯탕수육,푸실리파스타샐러드,풋고추,풋고추양파쌈장무침,풋마늘대무침,풋마늘초무침,풍기샐러드,프로바이오틱,피클,하루나겉절이,하와이안샐러드,하와이언함박스테이크,하이소스,한방갈비탕,한방설렁탕,한방소갈비찜,한식잡채,할라피뇨채소피클,함박스테이크,핫도그,해물겨자냉채,해물굴소스볶음,해물까스,해물누룽지탕,해물동그랑땡,해물동그랑땡채소볶음,해물돼지갈비찜,해물된장찌개,해물볶음우동,해물부추전,해물섞어찜,해물수제비국,해물순두부찌개,해물완자전,해물잡채,해물전,해물청경채볶음,해물콩나물찜,해물탕,해물파전,해장탕,해초무침,해초샐러드,해파리겨자채,해파리냉채,해파리무침,햄감자채볶음,햄계란말이,햄피망볶음,허니버터치킨,허니순살치킨,호두견과류강정,호박고추장찌개,호박꼬지,호박나물,호박된장국,호박된장찌개,호박맑은국,호박부추전,호박새우젓볶음,호박숙,호박잎,호박잎된장국,호박잎된장찌개,호박잎쌈,호박잎찌개,호박전,호박채나물,호박채나물볶음,홍삼,홍어무침,홍어미나리초무침,홍어채무침,홍초콩나물국,홍합국,홍합살무국,홍합살미역국,홍합탕,황도d,황도샐러드,황도요거트d,황태국,황태맑은국,황태미역국,황태양념구이,황태채국,황태채마늘쫑무침,황태콩나물해장국,황태포무침,후라이드,후르츠탕수육,훈제오리,훈제오리고추장볶음,훈제오리구이,훈제오리냉채,훈제오리단호박볶음,훈제오리마늘볶음,훈제오리볶음,흑임자d,흑임자드레싱,흑임자연근샐러드,히레카츠,금,목,수,월,화
0,2601,50,150,238,0.0,1039.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.364542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [17]:
lunch_df_test_f.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,d오리엔탈d,la갈비구이,new,가래떡구이,가래떡돼지갈비찜,가래떡츄러스,가자미구이,가자미무조림,가자미양념찜,가자미엿장구이,가자미엿장조림,가자미유린기,가자미조림,가자미카레튀김,가자미튀김,가지고추장무침,가지구이,가지나물,가지나물무침,가지두반장볶음,가지무침,가지볶음,가지완자튀김,가지찜,가쯔오국,가쯔오장국,간장,간장깻잎지,간장돼지갈비찜,간장두부조림,간장마늘치킨,간장양념,간장찜닭,간장치킨,간장파닭,갈릭돈가스,갈릭돈까스,갈릭버섯탕수,갈릭순살치킨,갈비만두,갈비탕,갈비통통만두,갈치감자조림,갈치구이,갈치무조림,갈치양념조림,갈치조림,감귤d,감자고구마샐러드,감자고추장찌개,감자국,감자그라탕,감자만두,감자범벅,감자볶음,감자비엔나볶음,감자샐러드,감자수제비국,감자양파국,감자전,감자조림,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자치즈구이,감자프리타타,강된장,강된장찌개,강정소스,개성감자만두,개성식메밀부침개,건강비빔밥,건다래순볶음,건새우마늘쫑볶음,건새우무국,건새우무채국,건새우미역국,건새우아욱국,건새우호박볶음,건새우호박채전,건취나물볶음,건파래무침,검은깨올방개묵무침,검정콩조림,견과류마카로니범벅,견과류멸치볶음,견과류샐러드,견과류연근조림,견과류조림,견과쌈장,경상도식소고기국,계란국,계란말이,계란버섯장조림,계란장조림,계란찜,계란채소볶음밥,계란파국,계란후라이,고구마고로케,고구마그라탕,고구마범벅,고구마순나물,고구마순무침,고구마순볶음,고구마줄기볶음,고구마치즈구이,고구마치즈돈까스,고구마치즈빵,고구마튀김,고구마함박스테이크,고기듬뿍카레라이스,고기완자전,고기전,고등어구이,고등어조림,고등어카레구이,고사리,고사리육개장,고추,고추가루,고추간장지,고추잎나물,고추잡채,고추장감자조림,고추장고구마순무침,고추장누들떡볶이,고추장불고기,고추장찌개,고추지,고추지무침,고추튀김,고춧잎볶음,곤드레밥,곤약메추리알조림,곤약야채무침,곤약어묵볶음,곤약흑임자무침,골뱅이무침,골뱅이소면무침,골뱅이야채무침,골뱅이채소무침,곰취,곰피초장,과일,과일그라탕,과일샐러드,과일요거트샐러드,과일탕수육,교촌간장치킨,구운채소,구이김,군고구마,군만두,굴떡국,굴미역국,굴비구이,궁중떡볶음,궁중떡볶이,궁중떡찜,그린샐러드,근대국,근대나물,근대된장국,김구이,김말이강정,김말이튀김,김밥볶음밥,김칫국,깍두기,깍둑오이초무침,깐풍기,깐풍두부,깐풍연근,깐풍육,깻순나물,깻잎,깻잎무쌈,깻잎순무침,깻잎쌈,깻잎양념지,깻잎완자전,깻잎전,깻잎지,깻잎찜,깻잎통닭,꼬들단무지무침,꼬막미나리초무침,꼬시래기무초무침,꼬시래기무침,꼬지삼색전,꼬지어묵탕,꽁치구이,꽁치레몬구이,꽁치오븐구이,꽁치와사비구이,꽁치한마리구이,꽁치한마리레몬구이,꽃게된장국,꽃게된장찌개,꽃게찌개,꽃게탕,꽃맛살과일샐러드,꽃맛살볶음,꽃맛살샐러드,꽃맛살오리엔탈샐러드,꽃맛살햄볶음,꽃빵,꽃상추겉절이,꽃상추무침,꽈리고추감자조림,꽈리고추메추리알조림,꽈리고추멸치볶음,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,나가사끼짬뽕국,나물비빔밥,나주곰탕,나쵸콥샐러드,낙지볶음,낙지볶음밥,낙지비빔밥,날치알계란찜,냉메밀소바,냉모밀국수,냉이국,냉이나물무침,냉이된장국,냉이된장찌개,냉이콩나물국,냉족발야채무침,너비아니구이,노각무침,노각생채,녹두전,누룽지탕수육,눈꽃치즈샐러드,느타리버섯국,느타리버섯볶음,늙은호박전,다대기,다슬기아욱국,...,치커리만다린샐러드,치커리무침,치커리사과무침,치커리사과생채,치커리사과유자청무침,치커리생채,치커리오이무침,치커리유자무침,치커리유자샐러드,치커리유자생채,치커리유자청무침,치커리유자청생채,치킨까스,치킨너겟,치킨무,치킨샐러드,치킨소스,치킨커틀렛,치킨텐더,치킨텐더샐러드,치킨핑거,칠리,칠리드레질,칠리새우,칠리소스,칠리탕수육,카레감자채볶음,카레닭볶음,카레닭찜,카레덮밥,카레라이스,카프레제샐러드,칼국수,캔꽁치튀김,케일,케일숙쌈,케찹,코다리강정,코다리양념조림,코다리조림,콘샐러드,콘소스d,콘슬로우,콘치즈오븐구이,콜리샐러드,콥샐러드,콩가루배추국,콩가루배추된장국,콩가루배춧국,콩나물겨자채,콩나물겨자채무침,콩나물국,콩나물김칫국,콩나물냉국,콩나물냉채,콩나물동태찜,콩나물두루치기,콩나물맑은국,콩나물맛살냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물부추무침,콩나물불고기,콩나물잡채,콩나물제육볶음,콩나물파채,콩나물파채무침,콩나물파채불고기,콩나물파채절이,콩비지찌개,콩샐러드,콩소스d,콩자반,콩조림,크란치바,크래미숙주무침,크래미해초무침,크레미계란말이,크루통샐러드,크리스마스케익,크림새우,크림스프,키위d,키위요거트d,타꼬야끼,타래과,타르,타르소스,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,탱크보이,토란국,토란탕,토마토,토마토계란볶음,토마토두부카프레제,토마토살사소스,토마토샐러드,토마토스크램블,토마토스파게티,토마토프리타타,톳두부무침,톳무침,통감자오븐구이,통계란꼬치어묵탕,통도라지고추장구이,통도라지구이,통들깨부추무침,통오이고추무침,파닭,파래김,파스타샐러드,파인d,파인애플,파인애플d,파인애플볶음밥,파인요거트d,파채,파채상추무침,파채콩나물무침,파프리카감자채볶음,파프리카계란말이,파프리카계란전,파프리카잡채,파프리카해초무침,팝콘치킨,팽이된장국,팽이무국,팽이버섯국,팽이장국,펜네파스타샐러드,포도d,포도주스,표고돈육탕수,표고버섯탕수육,푸실리파스타샐러드,풋고추,풋고추양파쌈장무침,풋마늘대무침,풋마늘초무침,풍기샐러드,프로바이오틱,피클,하루나겉절이,하와이안샐러드,하와이언함박스테이크,하이소스,한방갈비탕,한방설렁탕,한방소갈비찜,한식잡채,할라피뇨채소피클,함박스테이크,핫도그,해물겨자냉채,해물굴소스볶음,해물까스,해물누룽지탕,해물동그랑땡,해물동그랑땡채소볶음,해물돼지갈비찜,해물된장찌개,해물볶음우동,해물부추전,해물섞어찜,해물수제비국,해물순두부찌개,해물완자전,해물잡채,해물전,해물청경채볶음,해물콩나물찜,해물탕,해물파전,해장탕,해초무침,해초샐러드,해파리겨자채,해파리냉채,해파리무침,햄감자채볶음,햄계란말이,햄피망볶음,허니버터치킨,허니순살치킨,호두견과류강정,호박고추장찌개,호박꼬지,호박나물,호박된장국,호박된장찌개,호박맑은국,호박부추전,호박새우젓볶음,호박숙,호박잎,호박잎된장국,호박잎된장찌개,호박잎쌈,호박잎찌개,호박전,호박채나물,호박채나물볶음,홍삼,홍어무침,홍어미나리초무침,홍어채무침,홍초콩나물국,홍합국,홍합살무국,홍합살미역국,홍합탕,황도d,황도샐러드,황도요거트d,황태국,황태맑은국,황태미역국,황태양념구이,황태채국,황태채마늘쫑무침,황태콩나물해장국,황태포무침,후라이드,후르츠탕수육,훈제오리,훈제오리고추장볶음,훈제오리구이,훈제오리냉채,훈제오리단호박볶음,훈제오리마늘볶음,훈제오리볶음,흑임자d,흑임자드레싱,흑임자연근샐러드,히레카츠,금,목,수,월,화
0,2983,88,182,5,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [18]:
dinner_df_f.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,석식계,가래떡오븐구이,가쓰오장국,가자미구이,가자미양념조림,가자미양념찜,가자미엿장구이,가자미유린기,가자미찜,가자미카레튀김,가자미튀김,가정의날,가지나물,가지된장무침,가지마파두부,가지무침,가지탕수,가쯔오국,가쯔오장국,간장,간장계란장,간장깻잎지,간장마늘치킨,간장불고기,간장찜닭,간장치킨,갈릭파닭,갈비만두,갈치구이,갈치무조림,갈치양념조림,갈치조림,갈치카레구이,갈치튀김,감귤쥬스,감자간장조림,감자고추장찌개,감자국,감자만두,감자만두국,감자버터구이,감자범벅,감자비엔나볶음,감자샐러드,감자소세지볶음,감자양념조림,감자양파국,감자전,감자찌개,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자채햄볶음,감자치즈구이,감자치즈오븐구이,감자카레볶음,감자튀김,감자프리타타,감자햄조림,강된장,개성감자만두,건도토리묵파프리카볶음,건새우마늘쫑볶음,건새우무나물,건새우아욱국,건새우아욱된장국,건새우호박채전,건파래무침,건파래볶음,검정콩조림,겨울초겉절이,겨자장,견과류조림,계란국,계란김밥,계란떡볶이,계란말이,계란볶음밥,계란야채말이,계란장조림,계란찜,계란파국,계란후라이,계발의,고갈비구이,고구마그라탕,고구마까스,고구마떡볶이,고구마맛탕,고구마범벅,고구마샐러드,고구마순나물,고구마순들깨볶음,고구마순볶음,고구마연근맛탕,고구마오븐구이,고구마치즈구이,고구마치즈돈까스,고구마튀김,고기전,고기짬뽕국,고들빼기무침,고등어구이,고등어양념구이,고등어자반찜,고등어조림,고등어카레구이,고르곤졸라피자,고사리볶음,고추,고추가루,고추잡채,고추잡채덮밥,고추장멸치볶음,고추장불고기,고추장찌개,고추지무침,고추짜장,고추튀김,고춧잎무침,곤드레밥,골뱅이무침,골뱅이채소무침,과일,과일샐러드,과일요거트샐러드,과일주스,과일탕수육,과일플레인샐러드,교자만두,구슬떡볶이,구운채소,구이김,국내산,국물떡볶이,국산,군고구마,군만두,굴미역국,굴비구이,굴소스볶음밥,굴소스파인볶음밥,굴소스해물볶음밥,궁중떡볶음,궁중떡볶이,궁중떡찜,그린빈베이컨볶음,그린요거트샐러드,근대고추장무침,근대국,근대나물무침,근대된장국,근대두부된장국,김가루잔파무침,김계란말이,김구이,김말이,김말이강정,김말이튀김,김밥,김밥볶음밥,김자반,김자반볶음,김주먹밥,김칫국,까르보나라떡볶이,깍두기,깍둑오이초무침,깐쇼새우,깐풍기,깐풍연근,깐풍육,깨찰빵,깻순나물,깻잎고기전,깻잎쌈,깻잎양념찜,깻잎전,깻잎지,깻잎찜,꼬들단무지,꼬들단무지무침,꼬마김밥,꼬막찜,꼬막채소무침,꼬시래기무침,꼬시래기초무침,꼬지어묵국,꼬지어묵우동,꼬지어묵탕,꼬치어묵국,꼬치어묵떡볶이,꼬치어묵매운탕,꼬치어묵탕,꽁보리밥,꽁치감자조림,꽁치구이,꽁치양파조림,꽁치오븐구이,꽁치한마리구이,꽁치허브구이,꽃게탕,꽃맛살볶음,꽃맛살샐러드,꽃빵,꽃빵튀김,꽃상추무침,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,꽈배기도너츠,꿔바로우탕수육,나가사끼짬뽕,나가사끼짬뽕국,나가사키면,나가사키짬뽕국,나초콥샐러드,낙지덮밥,낙지미나리볶음,낙지볶음,낙지볶음덮밥,낙지비빔밥,낙지젓무침,날치알계란찜,날치알볶음밥,납작군만두,내사랑포도,냉메밀소바,냉모밀국수,냉이국,냉이김칫국,냉이된장국,냉이된장찌개,너비아니,노가리고추조림,노각무침,녹두빈대떡,녹두전,누룽지탕수육,뉴욕핫도그,느타리버섯국,느타리버섯볶음,느타리호박볶음,...,청포묵무침,청포묵야채무침,초간장,초계국수,초장,추가밥,춘권,춘장돈육볶음,춘전닭갈비,춘천닭갈비,충무김밥,취나물,취나물된장무침,취나물된장볶음,취나물땅콩무침,취나물무침,취나물볶음,츄러스,치즈계란말이,치즈계란찜,치즈닭갈비,치즈시즈닝치킨,치즈함박스테이크,치커리귤무침,치커리깨소스무침,치커리단감무침,치커리무침,치커리배생채,치커리사과무침,치커리사과생채,치커리생채,치커리유자무침,치커리유자청무침,치킨까스,치킨마요덮밥,치킨무,치킨샐러드,치킨퀘사디야,치킨텐더,치킨핑거,칠리s,칠리미트볼조림,칠리베이비크랩,칠리소스,카레감자채볶음,카레덮밥,카레돈까스정식,카레라이스,카레찜닭,카레홍합찜,칼국수,컬리플라워샐러드,컵라면,컵주스,케이준샐러드,케이준치킨샐러드,케일숙쌈,케찹,코다리강정,코다리무조림,코다리양념조림,코다리엿장조림,코다리조림,코코넛새우튀김,코코뱅,콘샐러드,콘슬로우,콘치즈,콘치즈구이,콘치즈오븐구이,콤비네이션피자,콥샐러드,콩가루배추국,콩국수,콩나물간장볶음,콩나물겨자채,콩나물국,콩나물굴소스볶음,콩나물냉국,콩나물냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물불고기,콩나물파채무침,콩비지찌개,콩자반,콩조림,퀘사디아,크래미오이보트샐러드,크랜베리단호박샐러드,크랜베리멸치볶음,크림새우,크림소스,크림스프,크림카레우동,클로렐라,키위그린샐러드,키위드레싱샐러드,타꼬야끼,타르타르,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,토란탕,토마토,토마토계란찜,토마토설탕절인,토마토스파게티,토마토시금치달걀볶음,톳두부무침,통고추쌈장무침,통도라지구이,통마늘너비아니조림,통배추겉절이,통새우김밥,통새우또띠아,투움바스파게티,파닭,파래김,파스타샐러드,파인애플볶음밥,파인애플주스,파채,파프리카해초무침,팝콘치킨,팥찜,팽이가쯔오장국,팽이버섯국,팽이버섯부추무침,팽이버섯장국,팽이버섯채소전,팽이장국,페퍼로니치즈피자,펜네베이컨샐러드,포기김치,포도,포도주스,포테이토오븐구이,폭탄주먹밥,표고버섯탕수육,푸딩,푸실리샐러드,풋고추,풋고추양파무침,풋고추튀김,풋마늘대무침,풋마늘초무침,풍기샐러드,피크닉,피클,하루나무침,하루야채주스,하와이안샐러드,하와이안필라프,한식잡채,할라피뇨,함박스테이크,핫도그,해라피겨자채,해물가스,해물겨자냉채,해물굴소스볶음밥,해물까스,해물누룽지,해물누룽지탕,해물동그랑땡,해물동그랑땡전,해물된장찌개,해물떡볶이,해물볶음,해물볶음밥,해물볶음우동,해물부추전,해물손수제비,해물수제비,해물순두부국,해물순두부찌개,해물우동볶음,해물전,해물짜장면,해물짬뽕,해물짬뽕국,해물청경채볶음,해물칼국수,해물파전,해쉬포테이토,해초레몬무침,해초무침,해초배무침,해파리겨자채,해파리냉채,햄감자볶음,햄감자채볶음,햄계란말이,햄맛살볶음,햄볶음밥,햄전,햄치즈또띠아,허니버터옥수수,허니슈스트링감자,호박나물,호박반달나물,호박부추전,호박새우젓국,호박잎된장찌개,호박잎찌개,호박젓국찌개,호박채나물,호빵,호주,홍시드레싱샐러드,홍어채무침,홍합무우국,홍합미역국,홍합짬뽕국,홍합탕,황태국,황태국수,황태무채국,황태미역국,황태양념구이,황태해장국,회오리감자,후난식볶음밥,후르츠탕수육,후리가케덮밥,훈제오리,훈제오리구이,훈제오리냉채,훈제오리떡볶음,훈제오리볶음,훈제오리불고기,흑미밥,흑임자곤약샐러드,흑임자드레싱,흑임자샐러드,흑임자시금치샐러드,금,목,수,월,화
0,2601,50,150,238,0.0,331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.549609,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [19]:
dinner_df_test_f.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,가래떡오븐구이,가쓰오장국,가자미구이,가자미양념조림,가자미양념찜,가자미엿장구이,가자미유린기,가자미찜,가자미카레튀김,가자미튀김,가정의날,가지나물,가지된장무침,가지마파두부,가지무침,가지탕수,가쯔오국,가쯔오장국,간장,간장계란장,간장깻잎지,간장마늘치킨,간장불고기,간장찜닭,간장치킨,갈릭파닭,갈비만두,갈치구이,갈치무조림,갈치양념조림,갈치조림,갈치카레구이,갈치튀김,감귤쥬스,감자간장조림,감자고추장찌개,감자국,감자만두,감자만두국,감자버터구이,감자범벅,감자비엔나볶음,감자샐러드,감자소세지볶음,감자양념조림,감자양파국,감자전,감자찌개,감자채볶음,감자채전,감자채카레볶음,감자채파프리카볶음,감자채햄볶음,감자치즈구이,감자치즈오븐구이,감자카레볶음,감자튀김,감자프리타타,감자햄조림,강된장,개성감자만두,건도토리묵파프리카볶음,건새우마늘쫑볶음,건새우무나물,건새우아욱국,건새우아욱된장국,건새우호박채전,건파래무침,건파래볶음,검정콩조림,겨울초겉절이,겨자장,견과류조림,계란국,계란김밥,계란떡볶이,계란말이,계란볶음밥,계란야채말이,계란장조림,계란찜,계란파국,계란후라이,계발의,고갈비구이,고구마그라탕,고구마까스,고구마떡볶이,고구마맛탕,고구마범벅,고구마샐러드,고구마순나물,고구마순들깨볶음,고구마순볶음,고구마연근맛탕,고구마오븐구이,고구마치즈구이,고구마치즈돈까스,고구마튀김,고기전,고기짬뽕국,고들빼기무침,고등어구이,고등어양념구이,고등어자반찜,고등어조림,고등어카레구이,고르곤졸라피자,고사리볶음,고추,고추가루,고추잡채,고추잡채덮밥,고추장멸치볶음,고추장불고기,고추장찌개,고추지무침,고추짜장,고추튀김,고춧잎무침,곤드레밥,골뱅이무침,골뱅이채소무침,과일,과일샐러드,과일요거트샐러드,과일주스,과일탕수육,과일플레인샐러드,교자만두,구슬떡볶이,구운채소,구이김,국내산,국물떡볶이,국산,군고구마,군만두,굴미역국,굴비구이,굴소스볶음밥,굴소스파인볶음밥,굴소스해물볶음밥,궁중떡볶음,궁중떡볶이,궁중떡찜,그린빈베이컨볶음,그린요거트샐러드,근대고추장무침,근대국,근대나물무침,근대된장국,근대두부된장국,김가루잔파무침,김계란말이,김구이,김말이,김말이강정,김말이튀김,김밥,김밥볶음밥,김자반,김자반볶음,김주먹밥,김칫국,까르보나라떡볶이,깍두기,깍둑오이초무침,깐쇼새우,깐풍기,깐풍연근,깐풍육,깨찰빵,깻순나물,깻잎고기전,깻잎쌈,깻잎양념찜,깻잎전,깻잎지,깻잎찜,꼬들단무지,꼬들단무지무침,꼬마김밥,꼬막찜,꼬막채소무침,꼬시래기무침,꼬시래기초무침,꼬지어묵국,꼬지어묵우동,꼬지어묵탕,꼬치어묵국,꼬치어묵떡볶이,꼬치어묵매운탕,꼬치어묵탕,꽁보리밥,꽁치감자조림,꽁치구이,꽁치양파조림,꽁치오븐구이,꽁치한마리구이,꽁치허브구이,꽃게탕,꽃맛살볶음,꽃맛살샐러드,꽃빵,꽃빵튀김,꽃상추무침,꽈리고추어묵볶음,꽈리고추어묵조림,꽈리고추찜,꽈리고추찹쌀무침,꽈배기도너츠,꿔바로우탕수육,나가사끼짬뽕,나가사끼짬뽕국,나가사키면,나가사키짬뽕국,나초콥샐러드,낙지덮밥,낙지미나리볶음,낙지볶음,낙지볶음덮밥,낙지비빔밥,낙지젓무침,날치알계란찜,날치알볶음밥,납작군만두,내사랑포도,냉메밀소바,냉모밀국수,냉이국,냉이김칫국,냉이된장국,냉이된장찌개,너비아니,노가리고추조림,노각무침,녹두빈대떡,녹두전,누룽지탕수육,뉴욕핫도그,느타리버섯국,느타리버섯볶음,느타리호박볶음,늙은호박전,...,청포묵무침,청포묵야채무침,초간장,초계국수,초장,추가밥,춘권,춘장돈육볶음,춘전닭갈비,춘천닭갈비,충무김밥,취나물,취나물된장무침,취나물된장볶음,취나물땅콩무침,취나물무침,취나물볶음,츄러스,치즈계란말이,치즈계란찜,치즈닭갈비,치즈시즈닝치킨,치즈함박스테이크,치커리귤무침,치커리깨소스무침,치커리단감무침,치커리무침,치커리배생채,치커리사과무침,치커리사과생채,치커리생채,치커리유자무침,치커리유자청무침,치킨까스,치킨마요덮밥,치킨무,치킨샐러드,치킨퀘사디야,치킨텐더,치킨핑거,칠리s,칠리미트볼조림,칠리베이비크랩,칠리소스,카레감자채볶음,카레덮밥,카레돈까스정식,카레라이스,카레찜닭,카레홍합찜,칼국수,컬리플라워샐러드,컵라면,컵주스,케이준샐러드,케이준치킨샐러드,케일숙쌈,케찹,코다리강정,코다리무조림,코다리양념조림,코다리엿장조림,코다리조림,코코넛새우튀김,코코뱅,콘샐러드,콘슬로우,콘치즈,콘치즈구이,콘치즈오븐구이,콤비네이션피자,콥샐러드,콩가루배추국,콩국수,콩나물간장볶음,콩나물겨자채,콩나물국,콩나물굴소스볶음,콩나물냉국,콩나물냉채,콩나물무침,콩나물밥,콩나물볶음,콩나물불고기,콩나물파채무침,콩비지찌개,콩자반,콩조림,퀘사디아,크래미오이보트샐러드,크랜베리단호박샐러드,크랜베리멸치볶음,크림새우,크림소스,크림스프,크림카레우동,클로렐라,키위그린샐러드,키위드레싱샐러드,타꼬야끼,타르타르,타르타르d,타르타르소스,타워함박스테이크,타코야끼,탄두리치킨,탈탈소스,탕수만두,탕수어,탕수육,탕평채,토란탕,토마토,토마토계란찜,토마토설탕절인,토마토스파게티,토마토시금치달걀볶음,톳두부무침,통고추쌈장무침,통도라지구이,통마늘너비아니조림,통배추겉절이,통새우김밥,통새우또띠아,투움바스파게티,파닭,파래김,파스타샐러드,파인애플볶음밥,파인애플주스,파채,파프리카해초무침,팝콘치킨,팥찜,팽이가쯔오장국,팽이버섯국,팽이버섯부추무침,팽이버섯장국,팽이버섯채소전,팽이장국,페퍼로니치즈피자,펜네베이컨샐러드,포기김치,포도,포도주스,포테이토오븐구이,폭탄주먹밥,표고버섯탕수육,푸딩,푸실리샐러드,풋고추,풋고추양파무침,풋고추튀김,풋마늘대무침,풋마늘초무침,풍기샐러드,피크닉,피클,하루나무침,하루야채주스,하와이안샐러드,하와이안필라프,한식잡채,할라피뇨,함박스테이크,핫도그,해라피겨자채,해물가스,해물겨자냉채,해물굴소스볶음밥,해물까스,해물누룽지,해물누룽지탕,해물동그랑땡,해물동그랑땡전,해물된장찌개,해물떡볶이,해물볶음,해물볶음밥,해물볶음우동,해물부추전,해물손수제비,해물수제비,해물순두부국,해물순두부찌개,해물우동볶음,해물전,해물짜장면,해물짬뽕,해물짬뽕국,해물청경채볶음,해물칼국수,해물파전,해쉬포테이토,해초레몬무침,해초무침,해초배무침,해파리겨자채,해파리냉채,햄감자볶음,햄감자채볶음,햄계란말이,햄맛살볶음,햄볶음밥,햄전,햄치즈또띠아,허니버터옥수수,허니슈스트링감자,호박나물,호박반달나물,호박부추전,호박새우젓국,호박잎된장찌개,호박잎찌개,호박젓국찌개,호박채나물,호빵,호주,홍시드레싱샐러드,홍어채무침,홍합무우국,홍합미역국,홍합짬뽕국,홍합탕,황태국,황태국수,황태무채국,황태미역국,황태양념구이,황태해장국,회오리감자,후난식볶음밥,후르츠탕수육,후리가케덮밥,훈제오리,훈제오리구이,훈제오리냉채,훈제오리떡볶음,훈제오리볶음,훈제오리불고기,흑미밥,흑임자곤약샐러드,흑임자드레싱,흑임자샐러드,흑임자시금치샐러드,금,목,수,월,화
0,2983,88,182,5,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.

## 중식 예측모델

In [20]:
lunch_regression_model = setup(data=lunch_df_f, target='중식계', train_size=0.8,n_jobs=-1,fold = 5)

,Description,Value
0,session_id,4488
1,Target,중식계
2,Original Data,"(1205, 1593)"
3,Missing Values,False
4,Numeric Features,881
5,Categorical Features,711
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 1163)"


In [21]:
# 최고 성능 모델 선정(2개)
lunch_regression_best_models = compare_models(fold=5, n_select=3, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,72.7938,9324.5101,96.1952,0.7827,0.1180,0.0880,2.4440
gbr,Gradient Boosting Regressor,72.6412,9543.5579,97.0932,0.7782,0.1201,0.0885,0.2500
lightgbm,Light Gradient Boosting Machine,74.1578,9573.2485,97.4287,0.7763,0.1205,0.0897,0.1400
rf,Random Forest Regressor,72.4099,9658.7841,97.7376,0.7750,0.1194,0.0873,0.3020
xgboost,Extreme Gradient Boosting,73.9969,10094.9518,99.9519,0.7650,0.1224,0.0895,0.6640
br,Bayesian Ridge,78.3454,10374.2384,101.5433,0.7581,0.1275,0.0956,1.1660
et,Extra Trees Regressor,77.0841,10800.6410,103.5038,0.7486,0.1245,0.0922,0.4620
ada,AdaBoost Regressor,82.6398,11407.9216,106.4965,0.7335,0.1297,0.1003,0.3000
en,Elastic Net,91.7826,13631.5850,116.6159,0.6812,0.1375,0.1088,0.4120
dt,Decision Tree Regressor,91.5978,15143.4702,122.4173,0.6486,0.1476,0.1092,0.0380


In [22]:
lunch_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in lunch_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,65.0474,6665.2556,81.6410,0.8257,0.0943,0.0756
1,74.7587,10194.1563,100.9661,0.7597,0.1199,0.0902
2,69.4038,8275.2061,90.9682,0.8031,0.1152,0.0860
3,71.7156,9277.5842,96.3202,0.8040,0.1286,0.0895
4,75.6712,11256.2960,106.0957,0.7436,0.1415,0.0965
Mean,71.3193,9133.6996,95.1982,0.7872,0.1199,0.0876
SD,3.8453,1579.9312,8.4258,0.0306,0.0156,0.0069


In [23]:
lunch_model_f = blend_models(estimator_list=lunch_regression_best_models_tuned, fold=5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,63.1074,6351.8641,79.6986,0.8339,0.0933,0.0738
1,74.3142,9423.8047,97.0763,0.7779,0.1147,0.0891
2,64.6812,6948.6220,83.3584,0.8347,0.1006,0.0781
3,69.8379,8951.4810,94.6123,0.8108,0.1248,0.0870
4,75.5731,11462.9304,107.0651,0.7389,0.1385,0.0950
Mean,69.5027,8627.7405,92.3621,0.7992,0.1144,0.0846
SD,4.9852,1831.7140,9.8478,0.0366,0.0163,0.0077


In [24]:
pred = predict_model(lunch_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,74.4554,9816.3988,99.0777,0.7948,0.1240,0.0922


### 테스트 데이터 예측

In [25]:
submission = pd.read_csv('sample_submission.csv')

In [26]:
final_model_lunch = finalize_model(lunch_model_f)
prep_pipe_lunch = get_config('prep_pipe')
prep_pipe_lunch.steps.append(['trained_model', final_model_lunch])

In [27]:
pred_lunch = prep_pipe_lunch.predict(lunch_df_test_f)

submission.iloc[:,1] = pred_lunch
submission.head()

,일자,중식계,석식계
0,2021-01-27,1014.787834,0
1,2021-01-28,928.621611,0
2,2021-01-29,659.541134,0
3,2021-02-01,1238.397852,0
4,2021-02-02,1005.653920,0


## 석식 예측모델

In [28]:
dinner_regression_model = setup(data=dinner_df_f, target='석식계', train_size=0.8,n_jobs=-1,fold = 5)

,Description,Value
0,session_id,6656
1,Target,석식계
2,Original Data,"(1205, 1585)"
3,Missing Values,False
4,Numeric Features,887
5,Categorical Features,697
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 1185)"


In [29]:
dinner_regression_best_models = compare_models(fold=5, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,67.5334,9539.5187,97.5068,0.5072,1.0323,0.1415,2.6460
rf,Random Forest Regressor,66.5913,9811.6077,98.7692,0.4945,1.0590,0.1353,0.3020
gbr,Gradient Boosting Regressor,68.4960,9899.4155,99.1810,0.4912,1.0744,0.1423,0.2460
xgboost,Extreme Gradient Boosting,72.1159,10628.0682,102.9239,0.4500,1.0558,0.1503,0.5720
en,Elastic Net,73.1011,11523.7107,106.9001,0.4134,1.1459,0.1436,0.0340
et,Extra Trees Regressor,71.3861,11454.6007,106.8848,0.4059,0.9450,0.1522,0.4600
br,Bayesian Ridge,73.9861,11771.5799,108.0428,0.4008,1.1489,0.1452,0.3260
lightgbm,Light Gradient Boosting Machine,72.7871,11764.8160,108.2349,0.3943,1.1250,0.1430,0.0640
knn,K Neighbors Regressor,73.2351,11978.3207,108.9090,0.3920,1.1397,0.1454,0.0760
ada,AdaBoost Regressor,90.4797,14086.4961,118.6673,0.2625,1.1002,0.1918,0.1200


In [30]:
dinner_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in dinner_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,74.1897,15295.0573,123.6732,0.3880,1.5679,0.1244
1,69.8939,11655.9161,107.9626,0.4276,1.2274,0.1326
2,59.6133,7544.6417,86.8599,0.4671,0.7643,0.1246
3,70.8246,12426.6327,111.4748,0.3942,1.2403,0.1366
4,67.8712,10946.0808,104.6235,0.3992,0.9876,0.1375
Mean,68.4785,11573.6657,106.9188,0.4152,1.1575,0.1311
SD,4.8803,2498.3983,11.9179,0.0293,0.2697,0.0057


In [31]:
dinner_model_f = blend_models(estimator_list=dinner_regression_best_models, fold=5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,65.6028,10703.2369,103.4564,0.5717,1.3781,0.1231
1,66.8771,8405.3909,91.6809,0.5873,1.0661,0.1412
2,62.1955,7328.5070,85.6067,0.4824,0.7274,0.1362
3,67.9328,10456.4832,102.2569,0.4902,1.1794,0.1360
4,66.7096,9829.9402,99.1461,0.4605,0.8723,0.1419
Mean,65.8636,9344.7116,96.4294,0.5184,1.0446,0.1357
SD,1.9772,1285.6499,6.7882,0.0511,0.2281,0.0068


In [32]:
pred = predict_model(dinner_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,56.1417,6335.5863,79.5964,0.6437,0.8237,0.1143


### 테스트 데이터 예측

In [33]:
final_model_dinner = finalize_model(dinner_model_f)
prep_pipe_dinner = get_config('prep_pipe')
prep_pipe_dinner.steps.append(['trained_model', final_model_dinner])

In [34]:
pred_dinner = prep_pipe_dinner.predict(dinner_df_test_f)
submission.iloc[:,2] = pred_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,1014.787834,382.264902
1,2021-01-28,928.621611,424.639996
2,2021-01-29,659.541134,273.191184
3,2021-02-01,1238.397852,504.196794
4,2021-02-02,1005.653920,418.869855


## 저장

In [35]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_pycaret.csv', index =False)

오늘 날짜 : 20210621
